# Ancilla Evolution - *After the System Thermalizes*

In [1]:
%matplotlib inline
import copy
from qutip import *
import numpy as np
import math, cmath
import matplotlib.pyplot as plt
from tqdm import tqdm

import utilities as use
from stateobj import QState, QAncilla

from IPython.display import Markdown, display

In [2]:
TIMESTEPS = 1000
TIMEDELTA = 1e-02
OMEGA = 10

## Thermal System

$$
\rho^* = \frac{1}{Z}
\begin{pmatrix}
    1 &  &  &  \\
    & \exp\left( \frac{\hbar\omega}{KT} \right) & & \\
    & & \exp\left( 2\frac{\hbar\omega}{KT} \right) & \\
    & & & \ddots \\
    & & & & \exp\left( n\frac{\hbar\omega}{KT} \right) \\
    & & & & & \ddots
\end{pmatrix}
= \frac{1}{Z}
\begin{pmatrix}
    E^0 &  &  &  \\
    & E^1 & & \\
    & & E^2 & \\
    & & & \ddots \\
    & & & & E^n \\
    & & & & & \ddots
\end{pmatrix}
$$

In [14]:
SIM_ID = '60D'
system = qload(f'objects/steadystate_meq_{SIM_ID}')
system

Loaded Qobj object:
Quantum object: dims = [[60], [60]], shape = (60, 60), type = oper, isHerm = True



Quantum object: dims = [[60], [60]], shape = (60, 60), type = oper, isherm = True
Qobj data =
[[1.10266496e-08 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.45690844e-08 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.92926877e-08 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.40625011e-01
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  1.87500014e-01 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 2.50000019e-01]]

## Ancilla State

In [4]:
alpha = complex(1/math.sqrt(2), 0)
beta = complex(1/math.sqrt(2), 0)
phi = np.pi/3
eta = QAncilla(alpha, beta, phi)
# Stampa informazioni sull'Ancella
display(eta)

Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
Qobj data =
[[0.5  +0.j         0.   +0.j         0.   +0.j        ]
 [0.   +0.j         0.25 +0.j         0.125+0.21650635j]
 [0.   +0.j         0.125-0.21650635j 0.25 +0.j        ]]

## Interaction

In [5]:
# System Opeators
ap_int = tensor(system.ap, qeye(3))
am_int = tensor(system.am, qeye(3))
# Ancilla Operators 
bp_int = tensor(qeye(system.dimentions), eta.sigmaminus)
bm_int = tensor(qeye(system.dimentions), eta.sigmaplus)
# INTERACTION
V = OMEGA * ( ap_int*bp_int + am_int*bm_int )

## Evolution

In [6]:
total_system = tensor(system, eta)
total_system = use.evolve(total_system, V, TIMEDELTA)
eta_new = total_system.ptrace(1)
eta_new

Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
Qobj data =
[[0.50000003+0.j         0.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.25000002+0.j         0.12500001+0.10620357j]
 [0.        +0.j         0.12500001-0.10620357j 0.25000002+0.j        ]]

## Theoretical Evolution
$$ \eta' = \eta + \Delta t\left[
   \langle A_+A_- \rangle\left(B_-\eta B_+ -\frac{1}{2}[B_+B_-; \eta]\right) +
   \langle A_-A_+ \rangle\left(B_+\eta B_- -\frac{1}{2}[B_-B_+; \eta]\right)
    \right] 
$$

### *Moments*

In [7]:
print(f"<A+> = {expect(system.ap, system).real}")
print(f"<A+> = {expect(system.ap, system).real}")

<A+> = 0.0
<A+> = 0.0


In [8]:
first_moments = expect([system.ap, system.am], system)
print(f"First Moments:\t\tComputed\tTheoretical\n"
      f"\tA- :\t\t{first_moments[0].real:.4f}\t\t0\n"
      f"\tA+ :\t\t{first_moments[1].real:.4f}\t\t0\n")
second_moments = expect([system.ap*system.ap, system.am*system.am], system)
print(f"Second Moments:\t\tComputed\tTheoretical\n"
      f"\tA-A- :\t\t{second_moments[0].real:.4f}\t\t0\n"
      f"\tA+A+ :\t\t{second_moments[1].real:.4f}\t\t0\n")


First Moments:		Computed	Theoretical
	A- :		0.0000		0
	A+ :		0.0000		0

Second Moments:		Computed	Theoretical
	A-A- :		0.0000		0
	A+A+ :		0.0000		0



### *Cross Correlations*
$$\text{Tr}\{a^\dagger a\rho^*\} = \frac{1}{Z}\sum_{n=0}^{\infty} nE^n \equiv \frac{1}{Z}\frac{E}{(1-E)^2}$$
$$\text{Tr}\{a a^\dagger\rho^*\} = \frac{1}{Z}\sum_{n=0}^{\infty} (n+1)E^n \equiv \frac{1}{Z}\frac{E}{(1-E)^2}+1$$

In [9]:
apam = expect(system.ap*system.am, system)
amap = expect(system.am*system.ap, system)
# Sum of infinite series
series = eta.factor/(system.partition-eta.factor)**2
# Discrete sum up to n
# number vector
n = np.arange(system.dimentions)
disc_sum = np.sum(n * system.diag())
disc_sump = np.sum((n+1) * system.diag())
print(f"Cross-correlations:\tComputed\tTheoretical\n"
      f"\tA+A- :\t\t{apam:>8.7}\t{disc_sum:>0.7}\n"
      f"\tA-A+ :\t\t{amap:>8.7}\t{disc_sump:>0.7}")

Cross-correlations:	Computed	Theoretical
	A+A- :		    56.0	56.0
	A-A+ :		    42.0	57.0


In [10]:
system.diag()[1]

1.4569084399051786e-08

In [11]:
eta.factor/system.partition

1.47021995005008e-08

In [12]:
np.sum(np.arange(len(system.diag()))*system.diag())

56.00000488255883

In [13]:
first_term = apam*(eta.sigmaminus*eta*eta.sigmaplus-0.5*commutator(eta.sigmaplus*eta.sigmaminus, eta))
second_term = amap*(eta.sigmaplus*eta*eta.sigmaminus-0.5*commutator(eta.sigmaminus*eta.sigmaplus, eta))
eta_new = eta + TIMEDELTA * (first_term + second_term)
eta_new

Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = False
Qobj data =
[[0.81500003+0.j         0.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.53000002+0.09093268j 0.40500002+0.21650635j]
 [0.        +0.j         0.40500002-0.21650635j 0.53000002-0.09093268j]]